In [2]:
import pandas as pd
import tensorflow
import os
from tensorflow import keras
import tensorflow as tf
import segmentation_models as sm
# import matplotlib.pyplot as plt
import numpy as np
from keras_unet_collection import models
import tensorflow_addons as tfa
import optuna
from optkeras.optkeras import OptKeras
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import MeanIoU
import kerastuner as kt
# from tensorflow import tensorflow.keras.mixed_precision.set_global_policy("mixed_float16")x

2023-07-18 16:08:09.965525: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 16:08:11.753507: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Segmentation Models: using `keras` framework.


/home/spotter5/.local/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/tmp/ipykernel_669006/366055844.py:18: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [ ]:
't'

Read in the training files

In [3]:

#get all the pathways
training_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_proj_0_128_training_files.csv')['Files'].tolist()
validation_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_proj_0_128_validation_files.csv')['Files'].tolist()
testing_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_proj_0_128_testing_files.csv')['Files'].tolist()

# #get negative name3s
# training_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/mtn_negative_images_crop_subs_128_training_files.csv')['Files'].tolist()
# validation_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/mtn_negative_images_crop_subs_128_validation_files.csv')['Files'].tolist()
# testing_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/mtn_negative_images_crop_subs_128_testing_files.csv')['Files'].tolist()

# training_names3 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/random_negative_images_subs_128_training_files.csv')['Files'].tolist()
# validation_names3 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/random_negative_images_subs_128_validation_files.csv')['Files'].tolist()
# testing_names3 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/random_negative_images_subs_128_testing_files.csv')['Files'].tolist()

# training_names = training_names + training_names2 + training_names3
# validation_names = validation_names + validation_names2 + validation_names3
# testing_names = testing_names + testing_names2 + testing_names3

min_max = pd.read_csv("/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_global_min_max_cutoff_proj.csv").reset_index(drop = True)

min_max = min_max[['6']]

len(training_names)


396841

Set up image generator

In [5]:
def data():
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    
    training_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_proj_0_128_training_files.csv')['Files']
    validation_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_proj_0_128_validation_files.csv')['Files']
    testing_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_proj_0_128_testing_files.csv')['Files']

#     training_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/mtn_negative_images_crop_subs_128_training_files.csv')['Files'].tolist()
#     validation_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/mtn_negative_images_crop_subs_128_validation_files.csv')['Files'].tolist()
#     testing_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/mtn_negative_images_crop_subs_128_testing_files.csv')['Files'].tolist()

#     training_names3 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/random_negative_images_subs_128_training_files.csv')['Files'].tolist()
#     validation_names3 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/random_negative_images_subs_128_validation_files.csv')['Files'].tolist()
#     testing_names3 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/random_negative_images_subs_128_testing_files.csv')['Files'].tolist()

#     training_names = training_names + training_names2 + training_names3
#     validation_names = validation_names + validation_names2 + validation_names3
#     testing_names = testing_names + testing_names2 + testing_names3

    min_max = pd.read_csv("/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_global_min_max_cutoff_proj.csv").reset_index(drop = True)

    min_max = min_max[['6']]

    #read in the trainiing and testing files and get lissts
    train_files = training_names[:100]
    val_files = validation_names[:100]
    test_files = testing_names[:100]

    #'Generates data containing batch_size samples' 
    
    #-------first get training data
    x_train = np.empty((len(train_files),128,128),  dtype="float32")
    y_train = np.empty((len(train_files),128,128,1),  dtype="uint8")
    
    # x_train = np.empty((101,128,128,9),  dtype="float32")
    # y_train = np.empty((101,128,128,1),  dtype="uint8")
    
    #start populating x by enumerating over the input img paths
    for j, path in enumerate(train_files):

        #load image
        img =  np.round(np.load(path), 3)[:, :, 6]

        # img = img * 1000
        img = img.astype(float)
        img = np.round(img, 3)
        img[img == 0] = -999

        img[np.isnan(img)] = -999


        img[img == -999] = np.nan

        in_shape = img.shape

        #turn to dataframe to normalize
        img = img.reshape(img.shape[0] * img.shape[1])

        img = pd.DataFrame(img)

        img.columns = min_max.columns

        img = pd.concat([min_max, img]).reset_index(drop = True)


        #normalize 0 to 1
        img = pd.DataFrame(scaler.fit_transform(img))

        img = img.iloc[2:]

        img = img.values.reshape(in_shape)

        img[np.isnan(img)] = -1



        img = np.round(img, 3)

        x_train[j] = img#[:, :, 4:] index number is not included, 

    for j, path in enumerate(train_files):

        #load image
        img =  np.round(np.load(path), 3)[:, :, -1]

        img = img.astype(int)

        img[img < 0] = 0
        img[img >1] = 0
        img[~np.isin(img, [0,1])] = 0

        img[np.isnan(img)] = 0
        img = img.astype(int)

        y_train[j] = np.expand_dims(img, 2) 

        
    x_val = np.empty((len(val_files),128,128),  dtype="float32")
    y_val = np.empty((len(val_files),128,128,1),  dtype="uint8")

    
    for j, path in enumerate(val_files):

        img =  np.round(np.load(path), 3)[:, :, 6]

        img = img.astype(float)
        img = np.round(img, 3)
        img[img == 0] = -999

        img[np.isnan(img)] = -999


        img[img == -999] = np.nan

        in_shape = img.shape

        img = img.reshape(img.shape[0] * img.shape[1])

        img = pd.DataFrame(img)

        img.columns = min_max.columns

        img = pd.concat([min_max, img]).reset_index(drop = True)


        img = pd.DataFrame(scaler.fit_transform(img))

        img = img.iloc[2:]
        img = img.values.reshape(in_shape)

        img[np.isnan(img)] = -1


        img = np.round(img, 3)
        x_val[j] = img#[:, :, 4:] index number is not included, 

    for j, path in enumerate(val_files):

        img =  np.round(np.load(path), 3)[:, :, -1]

        img = img.astype(int)

        img[img < 0] = 0
        img[img >1] = 0
        img[~np.isin(img, [0,1])] = 0

        img[np.isnan(img)] = 0
        img = img.astype(int)

        y_val[j] = np.expand_dims(img, 2) 
        
    x_test = np.empty((len(test_files),128,128),  dtype="float32")
    y_test = np.empty((len(test_files),128,128,1),  dtype="uint8")
    

    for j, path in enumerate(test_files):

        img =  np.round(np.load(path), 3)[:, :, 6]

        img = img.astype(float)
        img = np.round(img, 3)
        img[img == 0] = -999

        img[np.isnan(img)] = -999


        img[img == -999] = np.nan

        in_shape = img.shape

        img = img.reshape(img.shape[0] * img.shape[1])

        img = pd.DataFrame(img)

        img.columns = min_max.columns

        img = pd.concat([min_max, img]).reset_index(drop = True)


        img = pd.DataFrame(scaler.fit_transform(img))

        img = img.iloc[2:]
        img = img.values.reshape(in_shape)

        img[np.isnan(img)] = -1


        img = np.round(img, 3)

        x_test[j] = img#[:, :, 4:] index number is not included, 

    for j, path in enumerate(test_files):

        img =  np.round(np.load(path), 3)[:, :, -1]

        img = img.astype(int)

        img[img < 0] = 0
        img[img >1] = 0
        img[~np.isin(img, [0,1])] = 0

        img[np.isnan(img)] = 0
        img = img.astype(int)

        y_test[j] = np.expand_dims(img, 2) 
        
    return x_train,y_train,x_val,y_val,x_test,y_test







Read in the images based on the generator

In [6]:
# Free up RAM in case the model definition cells were run multiple times
tensorflow.keras.backend.clear_session()


In [7]:
input_height = 128
input_width = 128
num_channels = 1
n_labels = 1
hyperparameters = {
    'learning_rate': (1e-5, 1e-2, 'log-uniform'),
    'filters': [16, 32, 64],
}

def build_model(hp):
    model = models.unet_plus_2d((None, None, num_channels),
                               # filter_num=hp.Int('filter_num', min_value=16, max_value=256, step=16),
                               filter_num = [16,32,64, 128, 256],
                               activation=hp.Choice('activation', values=['ReLU', 'GELU', 'Snake']),
                               n_labels = n_labels,
                               stack_num_up = hp.Choice('stack_num_up', values = [1,2]),
                               stack_num_down =  hp.Choice('stack_num_down', values = [1,2]),
                               output_activation = 'Sigmoid',
                               batch_norm = True,
                               pool = hp.Choice('pool', values = [True,False]),
                               # unpool = hp.Choice('unpool', values = [True,False]),
                               unpool = False,
                               backbone = hp.Choice('backbone', values = ['EfficientNetB7', 'VGG19', 'ResNet152', 'DenseNet169']),
                               # backbone = 'EfficientNetB7'
                               weights = None,
                               deep_supervision = True
                               )
        
    optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


# model_unet_from_scratch = models.unet_3plus_2d((None, None, 1), filter_num= [16,32,64,128,256], #make smaller64, 128, 256, 512,[16, 32, 64, 128]
#                    n_labels=num_classes, 
#                    stack_num_down=2, stack_num_up=2, 
#                    activation='ReLU', 
#                    output_activation='Sigmoid', 
#                    batch_norm=True, pool=False, unpool=False, 
#                    backbone='EfficientNetB7', weights=None, 
#                    freeze_backbone=False, freeze_batch_norm=False, 
#                    deep_supervision = True,
#                    name='unet')


In [8]:
tuner = kt.BayesianOptimization(
    build_model,
    # objective= kt.Objective('val_mean_iou', direction="max"),  
    objective= 'val_loss',  

    max_trials=10,
    directory='my_directory',
    project_name='unet_tuning'
)

tuner.search_space_summary()

# Add the filter_num hyperparameter to the search space
# tuner.search_space.update({'filter_num': [16, 32, 64]})

2023-07-18 16:11:51.868293: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 16:11:55.486031: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31011 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0
2023-07-18 16:11:55.520477: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 31011 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8a:00.0, compute capability: 7.0


----------
deep_supervision = True
names of output tensors are listed as follows ("sup0" is the shallowest supervision layer;
"final" is the final output layer):

	xnet_output_sup0_activation
	xnet_output_sup1_activation
	xnet_output_sup2_activation
	xnet_output_final_activation
Search space summary
Default search space size: 6
activation (Choice)
{'default': 'ReLU', 'conditions': [], 'values': ['ReLU', 'GELU', 'Snake'], 'ordered': False}
stack_num_up (Choice)
{'default': 1, 'conditions': [], 'values': [1, 2], 'ordered': True}
stack_num_down (Choice)
{'default': 1, 'conditions': [], 'values': [1, 2], 'ordered': True}
pool (Choice)
{'default': 1, 'conditions': [], 'values': [1, 0], 'ordered': True}
backbone (Choice)
{'default': 'EfficientNetB7', 'conditions': [], 'values': ['EfficientNetB7', 'VGG19', 'ResNet152', 'DenseNet169'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


/home/spotter5/.local/lib/python3.10/site-packages/keras_unet_collection/_model_unet_plus_2d.py:246: UserWarning: 

The shallowest U-net++ deep supervision branch directly connects to a frozen backbone.
Testing your configurations on `keras_unet_collection.base.unet_plus_2d_base` is recommended.
  warnings.warn(backbone_warn);


In [9]:
 x_train,y_train,x_val,y_val,x_test,y_test = data()

In [10]:
x_train[0].shape

(128, 128)

In [ ]:
num_epochs = 10
tuner.search(x_train, y_train, epochs=num_epochs, validation_data=(x_val, y_val))

# Get the best model hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

Trial 5 Complete [00h 00m 29s]
val_loss: 11.866268157958984

Best val_loss So Far: 2.712986469268799
Total elapsed time: 00h 03m 09s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
Snake             |ReLU              |activation
1                 |2                 |stack_num_up
2                 |1                 |stack_num_down
1                 |1                 |pool
EfficientNetB7    |EfficientNetB7    |backbone
0.01              |0.0001            |learning_rate

----------
deep_supervision = True
names of output tensors are listed as follows ("sup0" is the shallowest supervision layer;
"final" is the final output layer):

	xnet_output_sup0_activation
	xnet_output_sup1_activation
	xnet_output_sup2_activation
	xnet_output_final_activation
Epoch 1/10


In [119]:
't'

't'

In [12]:
final = pd.DataFrame([best_hps.values])
print(final)

  activation  stack_num_up  stack_num_down  pool     backbone  learning_rate
0       ReLU             1               1     0  DenseNet169           0.01
